In [1]:
import openmm_scaled_md as scaled
import simtk.openmm as mm
from simtk.openmm import app
from simtk import unit

This will include several versions of setup, depending on where you're getting the initial conditions. You'll see that in all cases, the process of running the simulation is about the same. There are only small differences in how we set up the `system` object.

For more details on how to set up OpenMM simulations, see http://builder.openmm.org/.

### First version: Using `openmmtools.test_systems`

In [2]:
# load information about the system
import openmmtools
alanine = openmmtools.testsystems.AlanineDipeptideExplicit()

In [3]:
# this is the only place we use the openmm_scaled_md 
testsys_integrator = scaled.integrators.VVVRIntegrator(
    temperature=300.0*unit.kelvin,
    collision_rate=1.0/unit.picosecond,
    timestep=2.0*unit.femtosecond,
    force_scaling=0.6
)
# could also use LangevinIntegrator or BAOABIntegrator

In [4]:
# assemble the simulation object
testsys_sim = app.Simulation(topology=alanine.topology,
                             system=alanine.system,
                             integrator=testsys_integrator)
testsys_sim.context.setPositions(alanine.positions)
# output every 10th frame to DCD
testsys_sim.reporters.append(app.DCDReporter('testsys.dcd', 10))

Note that this cheats a little, because really we should energy minimize/equilibrate, but we don't bother here. Run for 1000 steps, giving 100 frames.

In [5]:
testsys_sim.step(1000)

### Second version: Using a PDB file

This assumes that the PDB represents solvated and equilibrated initial conditions.

Download `AD_initial_frame.pdb` from: https://raw.githubusercontent.com/openpathsampling/openpathsampling/master/examples/resources/AD_initial_frame.pdb

In [6]:
pdb = app.PDBFile('AD_initial_frame.pdb')
forcefield = app.ForceField('amber96.xml', 'tip3p.xml')

pdb_system = forcefield.createSystem(pdb.topology,
                                     nonbondedMethod=app.PME, 
                                     nonbondedCutoff=1.0*unit.nanometers,
                                     constraints=app.HBonds,
                                     rigidWater=True,
                                     ewaldErrorTolerance=0.0005)

In [7]:
# this is the only place we use the openmm_scaled_md 
pdb_integrator = scaled.integrators.VVVRIntegrator(
    temperature=300.0*unit.kelvin,
    collision_rate=1.0/unit.picosecond,
    timestep=2.0*unit.femtosecond,
    force_scaling=0.6
)
# could also use BAOABIntegrator

In [8]:
pdb_sim = app.Simulation(pdb.topology, pdb_system, pdb_integrator)
pdb_sim.context.setPositions(pdb.positions)
pdb_sim.reporters.append(app.DCDReporter('from_pdb.dcd', 10))

In [9]:
pdb_sim.step(1000)

### Third version: Using a Gromacs `.gro` file

In [10]:
gro = app.GromacsGroFile('AD_initial_frame.gro')
box_vectors = mm.Vec3(*gro.getPeriodicBoxVectors())
top = app.GromacsTopFile('AD.top', periodicBoxVectors=box_vectors)

In [11]:
gro_system = top.createSystem(nonbondedMethod=app.PME,
                              nonbondedCutoff=1.0*unit.nanometers,
                              constraints=app.HBonds,
                              rigidWater=True,
                              ewaldErrorTolerance=0.0005)

In [12]:
# this is the only place we use the openmm_scaled_md 
gro_integrator = scaled.integrators.VVVRIntegrator(
    temperature=300.0*unit.kelvin,
    collision_rate=1.0/unit.picosecond,
    timestep=2.0*unit.femtosecond,
    force_scaling=0.6
)
# could also use LangevinIntegrator or BAOABIntegrator

In [13]:
gro_sim = app.Simulation(top.topology, gro_system, gro_integrator)
gro_sim.context.setPositions(gro.positions)
gro_sim.reporters.append(app.DCDReporter('from_gro.dcd', 10))

In [14]:
gro_sim.step(1000)